<a href="https://colab.research.google.com/github/AbdullahAboElHija/CloudCourse/blob/main/Hw3_211575790_207976382_323016535_322253758_212209670.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install firebase

In [ ]:
try:
    from fuzzywuzzy import process
except ImportError:
    !pip install fuzzywuzzy[speedup]

import requests
"""
  Implementing the text Proccess Using Stemming and fuzzywuzzy
  Helper Function
"""

# DB_URL = "https://cloud-project-38ea3-default-rtdb.firebaseio.com/"
DB_URL = "https://razicloudlearn-default-rtdb.europe-west1.firebasedatabase.app/"

# get all the terms from DB
def getAllTermsFromDB(db_url):
  url = f"{db_url}/index.json"
  response = requests.get(url)
  if response.status_code != 200:
    return []

  all_data = response.json()
  terms = []
  for unique_key in all_data:
    if 'term' in all_data[unique_key]:
      terms.append(all_data[unique_key]['term'])
  return terms

def test():
  return 1




from fuzzywuzzy import process
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk

# Ensure NLTK is ready with necessary components
nltk.download('punkt')
nltk.download('wordnet')

# Initialize the stemmer
stemmer = PorterStemmer()

"""
  Main Function to do the job

"""

def findClosestTerms(query, db_url, threshold=80):
    indexed_terms = getAllTermsFromDB(db_url)
    indexed_stems = {term: stemmer.stem(term) for term in indexed_terms}

    query_tokens = word_tokenize(query.lower())
    query_stems = [stemmer.stem(word) for word in query_tokens]

    # Prepare a list for matching scores
    matches = []

    # For each stemmed query token, find the best match in indexed stems
    for query_stem in query_stems:
        # Use fuzzy matching to find the closest indexed term for each stem in the query
        closest_match, score = process.extractOne(query_stem, indexed_stems.values())

        # If the score meets the threshold, find the original term for the closest match
        if score >= threshold:
            for term, stem in indexed_stems.items():
                if stem == closest_match:
                    matches.append((term, score))
                    break

    # Return unique terms that matched, sorted by score
    unique_matches = sorted(set(matches), key=lambda x: x[1], reverse=True)
    return ' '.join([match[0] for match in unique_matches])



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
import requests
"""
TERMS = ["cloud", "saas", "database", "docker", "python", "computing", "paas", "iaas",
         "api", "container", "provider", "service", "technology", "oracle", "flexibility",
         "scalability", "application"]
"""
DB_URL = "https://razicloudlearn-default-rtdb.europe-west1.firebasedatabase.app/"

"""
  Helper functions
"""

def getAllTermsWithKeys(db_url):
    # Fetch the entire index
    url = f"{db_url}/index.json"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {}

def findTermData(term, db_url):
    all_terms = getAllTermsWithKeys(db_url)
    for unique_key in all_terms:
        if 'term' in all_terms[unique_key] and all_terms[unique_key]['term'] == term:
            return all_terms[unique_key]['DocIDs']
    return {}


"""
  Implementing the code for the user search

"""

def searchAndRankLinks(user_query, db_url):
    if len(user_query) > 50:
      return []
    proccessed_query_words = findClosestTerms(user_query, db_url) # string with closes relevant words
    query_words = proccessed_query_words.lower().split()

    # init a dictionary to hold link scores
    link_scores = {}

    # Process each word in the query
    for word in query_words:
        term_data = findTermData(word, db_url)
        if not term_data:
          continue

        for link_frequency_pair in term_data:
          link = link_frequency_pair[0]
          frequency = link_frequency_pair[1]
          if link in link_scores:
             link_scores[link] += frequency
          else:
            link_scores[link] = frequency

    # Sort links by their scores, return a list of tuples
    ranked_links = sorted(link_scores.items(), key=lambda item: item[1], reverse=True)

    return ranked_links



In [ ]:
# HTML code for the search results screen
search_results_html = '''
<!DOCTYPE html>
<html>
<head>
    <title>Search Results</title>

    <script>
      function updateSearchResults(html) {
          // Clear previous results and update with new HTML
          const resultsDiv = document.getElementById('results');
          resultsDiv.innerHTML = html; // This replaces the current contents with new HTML
      }
    </script>

    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 20px;
            background-color: #f0f0f0;
        }
        h2 {
            text-align: center;
        }
        .results {
            margin-top: 20px;
        }
        .result {
            margin-bottom: 10px;
        }
        .result a {
            color: #007bff;
            text-decoration: none;
        }
    </style>
</head>
<body>
    <h2>Search Results</h2>
    <div class="results" id="results">
        <!-- Search results will be displayed here -->
    </div>
</body>
</html>
'''

In [ ]:
search_bar_html = '''
<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Search Bar</title>
    <style>
         body {
          font-family: Arial, sans-serif;
          margin: 0;
          padding: 0;
          position: absolute;
          top: 50%;
          left: 50%;
          transform: translate(-50%, -50%);
          height: 100vh; /* Set height to 100% of viewport height */
          background-color: #f0f0f0;
}
        .container {
            text-align: center;
        }
        input[type="text"] {
            padding: 10px;
            width: 300px;
            border-radius: 5px;
            border: 1px solid #ccc;
            margin-right: 10px;
        }
        input[type="submit"] {
            padding: 10px 20px;
            background-color: #007bff;
            color: #fff;
            border: none;
            border-radius: 5px;
            cursor: pointer;
        }

            #search-results {
              list-style-type: none;
              padding: 0;
              width: 300px;
              margin: 0;
              border-radius: 5px;
              border: 1px solid #ccc;
}

        #search-results.loading::before {
          content: 'Loading...';
          display: block;
          padding: 8px;
          text-align: center;
          color: #fffcfe; /* Adjust the color as needed */}

          #search-results li {
            list-style: none;
            padding: 8px;
            cursor: pointer;
            transition: background-color 0.3s ease;
             background-color: #fffcfe;
            }
            #search-results li:hover {
              background-color: #f2f1e9; /* Adjust the hover background color as needed */}


    </style>
    <script src="https://cdn.tailwindcss.com"></script>
</head>
<body>
    <div>
    <nav class=" border-b-4 border-[#007bff] p-2 ">
    <div class="container mx-auto flex justify-between items-center">
      <div
        class="container mx-auto flex items-center justify-between gap-8 relative"
      >
        <button
          id="menuBtn"
          class="block sm:hidden bg-white text-blue-500 py-2 px-4 rounded-full hover:bg-blue-100 ml-auto"
        >
          &#9776;
          <!-- Hamburger Icon -->
        </button>

          <div class="sm:flex space-x-4 hidden mt-2">
            <a href="#" onclick="handleHistoryClick(event)" class="hover:text-gray-300 focus:outline-none">History</a>
            <a href="#" onclick="handleBookmarksClick(event)" class="hover:text-gray-300 focus:outline-none">Bookmarks</a>
            <a href="#" onclick="handleLogoutClick(event)" class="hover:text-gray-300 focus:outline-none">Logout</a>
          </div>

        <div class="items-center sm:flex mr-2">


        </div>


      </div>
    </div>
  </nav>
    </div>

    <div class="container">

       <div class="search-container">
       <h2 class="text-orange-600 text-3xl p-4 mt-16 mb-4">Oracle CloudSeeker</h2>
        <form id="searchForm" class="pb-4">
           <input type="text" id="query" name="query" placeholder="Enter your search query">
           <input type="submit" value="Search">
           <input type="submit" id="luckyButton" value="I'm Feeling Lucky">
           <ul id="search-results"></ul>


        </form>
    </div>

    </div>

    <script>
        document.getElementById("searchForm").onsubmit = function(event) {
            event.preventDefault();
            var query = document.getElementById("query").value;
            google.colab.kernel.invokeFunction('notebook.search', [query], {});
        };

        document.getElementById("luckyButton").onclick = function(event) {
            event.preventDefault();
            document.getElementById("query").value = '';
            google.colab.kernel.invokeFunction('notebook.lucky', [], {});
        };
////////////////////////////////////////////////////////////////////////////////////////////

///auto complete
         function autocomplete() {
        const input = document.getElementById('query').value.toLowerCase();
        const resultsContainer = document.getElementById('search-results');
        resultsContainer.innerHTML = ''; // Clear previous results

        if (input.trim() !== '') {
            // use api
            fetch(`https://api.datamuse.com/sug?s=${input}`)
                .then(response => response.json())
                .then(data => {
                    data.forEach(item => {
                        const li = document.createElement('li');
                        li.textContent = item.word;
                        li.addEventListener('click', () => {
                            document.getElementById('query').value = item.word;
                            resultsContainer.style.display = 'none';
                        });
                        resultsContainer.appendChild(li);
                    });
                    resultsContainer.style.display = data.length > 0 ? 'block' : 'none';
                })
                .catch(error => console.error('Error fetching suggestions:', error));
        } else {
            resultsContainer.style.display = 'none';
        }
    }

    // Attach the event listener after the page has loaded
  document.getElementById('query').addEventListener('input', autocomplete);

  // Add click event listener to close suggestions on document click
  document.addEventListener('click', function (event) {
    const searchInput = document.getElementById('query');
    const resultsContainer = document.getElementById('search-results');

    if (event.target !== searchInput && !searchInput.contains(event.target) && !resultsContainer.contains(event.target)) {
      resultsContainer.style.display = 'none';
    }
  });





   function handleHistoryClick(event) {
    event.preventDefault();
    google.colab.kernel.invokeFunction('notebook.go_to_history_page', [], {});
}

    function handleBookmarksClick(event) {
      event.preventDefault();
      google.colab.kernel.invokeFunction('notebook.go_to_bookmark', [], {});
    }

    function handleLogoutClick(event) {
      event.preventDefault();
      google.colab.kernel.invokeFunction('notebook.go_to_logout', [], {});
    }
    </script>
</body>
</html>
'''

In [ ]:

login_screen_html = '''
<!DOCTYPE html>
<html>
<head>
    <title>Login</title>
    <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">
      <script src="https://cdn.tailwindcss.com"></script>
</head>
<body class="bg-gray-100 h-screen flex justify-center items-center ">

    <div class="w-full max-w-xs">
    <div>
      <h2 class="text-orange-600 text-3xl p-4 mt-16 mb-4">Oracle CloudSeeker</h2>
      </div>
        <form id="loginForm" class="bg-white shadow-md rounded px-8 pt-6 pb-8 mb-4">
            <div class="text-center text-xl text-bold pb-4" >Login User</div>
            <div class="mb-4">
                <label class="block text-gray-700 text-sm font-bold mb-2" for="username">
                    Username
                </label>
                <input class="shadow appearance-none border rounded w-full py-2 px-3 text-gray-700 leading-tight focus:outline-none focus:shadow-outline" id="username" type="text" placeholder="Username">
            </div>
            <div class="mb-6">
                <label class="block text-gray-700 text-sm font-bold mb-2" for="password">
                    Password
                </label>
                <input class="shadow appearance-none border  rounded w-full py-2 px-3 text-gray-700 mb-3 leading-tight focus:outline-none focus:shadow-outline" id="password" type="password" placeholder="Password">
            </div>
            <div class="flex items-center justify-between">
                <button class="bg-blue-500 hover:bg-blue-700 text-white font-bold py-2 px-4 rounded focus:outline-none focus:shadow-outline" type="button"  onclick="login_user()">
                    Login
                </button>
                <!-- Add the register button here -->
                <button class="  text-blue font-bold py-2 px-4 font-italic focus:outline-none focus:shadow-outline" type="button" onclick="goToRegister()">
                    Not registered?
                </button>

            </div>
        </form>
    </div>

    <script>
        function login_user() {
            var username = document.getElementById('username').value;
            var password = document.getElementById('password').value;
            // Make a request to login the user
            if (username === '') {
              alert('Please enter your username.');
              return; // Prevent further execution of the function
            }
            if (password === '') {
              alert('Please enter your password.');
              return; // Prevent further execution of the function
            }
            google.colab.kernel.invokeFunction('notebook.login_user', [username, password], {})
        }

        // Function to navigate to the register screen
        function goToRegister() {
            google.colab.kernel.invokeFunction('notebook.go_to_register', [], {});
        }
        function goToAdmin() {
            google.colab.kernel.invokeFunction('notebook.go_to_admin', [], {});
        }
    </script>
</body>
</html>
'''

In [ ]:
register_screen_html = '''
<!DOCTYPE html>
<html>
<head>
    <title>Register</title>
    <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">
      <script src="https://cdn.tailwindcss.com"></script>
</head>
<body class="bg-gray-100 h-screen flex justify-center items-center">
<h2 class="text-orange-600 text-3xl p-4 mt-16 mb-4">Oracle CloudSeeker</h2>
    <div class="w-full max-w-xs">
        <form id="registerForm" class="bg-white shadow-md rounded px-8 pt-6 pb-8 mb-4">
        <div class="text-xl text-bold text-center pb-4">Register User </div>
            <div class="mb-4">
                <label class="block text-gray-700 text-sm font-bold mb-2" for="username">
                    Username
                </label>
                <input class="shadow appearance-none border rounded w-full py-2 px-3 text-gray-700 leading-tight focus:outline-none focus:shadow-outline" id="username" type="text" placeholder="Username">
            </div>
            <div class="mb-6">
                <label class="block text-gray-700 text-sm font-bold mb-2" for="password">
                    Password
                </label>
                <input class="shadow appearance-none border  rounded w-full py-2 px-3 text-gray-700 mb-3 leading-tight focus:outline-none focus:shadow-outline" id="password" type="password" placeholder="Password">
            </div>
            <div class="flex items-center justify-between">
                <button class="bg-blue-500 hover:bg-blue-700 text-white font-bold py-2 px-4 rounded focus:outline-none focus:shadow-outline" type="button"  onclick="register_user()">
                    Register
                </button>
                <!-- Add the button to navigate back to the login screen -->
                <button class=" font-bold py-2 px-4 font-italic focus:outline-none focus:shadow-outline" type="button" onclick="goToLogin()">
                    Back to Login
                </button>
            </div>
        </form>
    </div>

    <script>
        function register_user() {
            var username = document.getElementById('username').value;
            var password = document.getElementById('password').value;
            if (username === '') {
              showAlert('Please enter your username.');
              return; // Prevent further execution of the function
            }
            if (password === '') {
              showAlert('Please enter your password.');
              return; // Prevent further execution of the function
            }
            // Make a request to add the new user
            google.colab.kernel.invokeFunction('notebook.register_user', [username, password], {});
            google.colab.kernel.invokeFunction('notebook.go_to_login', [], {});
        }

        // Function to navigate back to the login screen
        function goToLogin() {
            google.colab.kernel.invokeFunction('notebook.go_to_login', [], {});
        }

        function showAlert(message) {
                alert(message);
            }
    </script>
</body>
</html>
'''


# **ChatBot HTML CONTENT**

In [ ]:
htmlContentForChatBot = """
<!DOCTYPE html>
<html>
<head>
    <title>Chat Window</title>
    <!-- Load TailwindCSS from CDN -->
    <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">
</head>
<body>
  <p class="text-center mt-8">To calculate the ranking for each term's associated links, we begin by preprocessing the user's query. Next, leveraging our index, we divide the query into individual words and retrieve all corresponding URLs. For each URL, we examine the frequency of each word within the content, aggregating these frequencies for every link. The aggregate frequency serves as the site's rank.</p>
  <div class="max-w-3xl mx-auto text-center mt-8">
    <h1 class="text-4xl font-bold text-gray-900 leading-tight mb-2 pb-4 relative">
        <span class="bg-clip-text text-transparent bg-gradient-to-r from-purple-500 to-pink-500">Zebra's ChatBot</span>
        <span class="absolute bottom-0 left-0 w-full h-1 bg-gradient-to-r from-purple-500 to-pink-500"></span>
    </h1>
    <p class="text-lg text-gray-800 mb-4">Welcome to Zebra's ChatBot, we put alot of effort in creating this chatbot, I hope you enjoy it!</p>
  </div>
  <div class="flex flex-col h-4/6">
      <!-- Chat container -->
      <div id="chat-container" class="flex flex-col overflow-auto p-4 space-y-2 bg-gray-100 flex-grow">
          <!-- Messages will be appended here -->
      </div>

      <!-- Input area -->
      <div class="p-4 bg-gray-200">
          <input id="message-input" type="text" placeholder="Type your message here..." class="rounded border-gray-300 p-2 w-full">
          <button onclick="send_message(null)" class="bg-blue-500 hover:bg-blue-700 text-white font-bold py-2 px-4 rounded">
              Send
          </button>
      </div>
  </div>

  <script>
    // Function to append messages to the chat container

     function appendMessage(message, sender = 'user') {
        // Create a div element for the message
        var messageDiv = document.createElement("div");
        messageDiv.textContent = message;
        // Add sender specific styling here if desired
        messageDiv.className = sender === 'user' ? "p-2 bg-white rounded shadow" : "p-2 bg-green-100 rounded shadow";

        // Append the message to the chat container
        document.getElementById("chat-container").appendChild(messageDiv);

        // Scroll to the bottom of the chat container
        var chatContainer = document.getElementById("chat-container");
        chatContainer.scrollTop = chatContainer.scrollHeight;
    }

    function robotResponse(url, message_for_url, header_message, paragraph_message){
      // Create a div element for the message
      var messageDiv = document.createElement("div");
      // Add sender specific styling here if desired
      messageDiv.className = "p-2 bg-green-100 rounded shadow";

      // Create and append the message_for_url in a <p>
      var pMessageForUrl = document.createElement("p");
      pMessageForUrl.textContent = message_for_url;
      pMessageForUrl.className = "text-sm mb-2 text-gray-700"
      messageDiv.appendChild(pMessageForUrl);

      // Create and append the url in a <a>
      var aUrl = document.createElement("a");
      aUrl.setAttribute("href", url);
      aUrl.className = "text-blue-500 hover:text-blue-600 underline";
      aUrl.textContent = url;
      messageDiv.appendChild(aUrl);

      // Create and append the header_message in a <h1>
      var h1HeaderMessage = document.createElement("h1");
      h1HeaderMessage.className = "text-xl font-semibold text-gray-900 my-2";
      h1HeaderMessage.textContent = header_message;
      messageDiv.appendChild(h1HeaderMessage);

      // Create and append the paragraph_message in a <p>
      var pParagraphMessage = document.createElement("p");
      pParagraphMessage.textContent = paragraph_message;
      messageDiv.appendChild(pParagraphMessage);

      // Append the messageDiv to the chat container
      document.getElementById("chat-container").appendChild(messageDiv);
    }


    // Function to send and display messages
    function send_message(message, sender = 'user') {
        if(sender != 'user'){
          appendMessage(message, sender);
          return;
        }
        const input = document.getElementById("message-input");
        const message_from_user = input.value.trim();
        if(message_from_user !== ""){
          appendMessage(message_from_user);
          google.colab.kernel.invokeFunction('notebook.pipeMessageFromUserIntoChatBot', [message_from_user], {});
        }
        input.value = "";
    }

    document.getElementById("message-input").addEventListener("keyup", function(event) {
      if (event.keyCode === 13) { // 13 is the Enter key
          send_message();
      }
    });

    function openInNewTab(url) {
            var win = window.open(url, '_blank');
            win.focus();
    }

  </script>

</body>
</html>"""


In [ ]:
import requests
from bs4 import BeautifulSoup

# Function to fetch the HTML content of a given URL and parse it using BeautifulSoup
def fetch_page(url):
    """
    Fetches the HTML content of the given URL and returns a BeautifulSoup object.

    Args:
    url (str): The URL of the web page to fetch.

    Returns:
    BeautifulSoup object: Parsed HTML content if successful, otherwise None.
    """
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    return None

import re

# Function to create an index of words and their frequencies from parsed HTML content
def index_words(soup):
    """
    Creates an index of words and their frequencies from parsed HTML content.

    Args:
    soup (BeautifulSoup object): Parsed HTML content.

    Returns:
    dict: A dictionary containing words as keys and their frequencies as values.
    """
    index = {}
    words = re.findall(r'\w+', soup.get_text())
    for word in words:
        word = word.lower()
        if word in index:
            index[word] += 1
        else:
            index[word] = 1
    return index

from nltk.stem import PorterStemmer

# Function to apply stemming to the words in the index
def apply_stemming(index):
    """
    Applies stemming to the words in the index.

    Args:
    index (dict): A dictionary containing words as keys and their frequencies as values.

    Returns:
    dict: A dictionary containing stemmed words as keys and their frequencies as values.
    """
    stemmer = PorterStemmer()
    stemmed_index = {}
    for word, count in index.items():
        stemmed_word = stemmer.stem(word)
        if stemmed_word in stemmed_index:
            stemmed_index[stemmed_word] += count
        else:
            stemmed_index[stemmed_word] = count
    return stemmed_index

# Function to search for words in the index
def search(query, index):
    """
    Searches for words in the index.

    Args:
    query (str): The search query.
    index (dict): A dictionary containing words as keys and their frequencies as values.

    Returns:
    dict: A dictionary containing matching words from the query and their frequencies.
    """
    stemmer = PorterStemmer()
    query_words = re.findall(r'\w+', query.lower())
    results = {}
    for word in query_words:
        word = stemmer.stem(word)
        if word in index:
            results[word] = index[word]
    return results

# Function to remove stop words from the index
def remove_stop_words(index):
    """
    Removes stop words from the index.

    Args:
    index (dict): A dictionary containing words as keys and their frequencies as values.

    Returns:
    dict: A dictionary with stop words removed.
    """
    stop_words = ['a', 'an', 'the', 'and', 'or', 'in', 'on', 'at']
    cleaned_index = {word: count for word, count in index.items() if word not in stop_words}
    return cleaned_index

from collections import Counter

# Function to calculate word frequencies inside the text
def wordsFrequencyInsideText(words, soup):
    """
    Calculates word frequencies inside the text of the parsed HTML content.

    Args:
    words (list): List of words to search for.
    soup (BeautifulSoup object): Parsed HTML content.

    Returns:
    dict: A dictionary containing the frequencies of specified words found in the text.
    """
    soup_text = soup.get_text().lower()
    word_counts = Counter(soup_text.split())
    word_freq = {}
    for word in words:
        count = word_counts.get(word, 0)
        if count > 0:
            word_freq[word] = count
    return word_freq

# Function to search for specified words in a URL
def search_for_words_in_url(url, words):
    """
    Searches for specified words in the content of the given URL.

    Args:
    url (str): The URL of the web page to search.
    words (list): List of words to search for.

    Returns:
    dict: A dictionary containing the frequencies of specified words found in the URL content.
    """
    soup = fetch_page(url)
    if not soup:
        return soup
    index = index_words(soup)
    index = remove_stop_words(index)
    index = apply_stemming(index)
    merge_words = ' '.join(words)
    words_frequencies = search(merge_words, index)
    return words_frequencies

# Function to search for specified words in a URL with optimized approach
def search_for_words_in_url_optimized(url, words):
    """
    Searches for specified words in the content of the given URL using an optimized approach.

    Args:
    url (str): The URL of the web page to search.
    words (list): List of words to search for.

    Returns:
    dict: A dictionary containing the frequencies of specified words found in the URL content.
    """
    soup = fetch_page(url)
    if not soup:
        return soup
    return wordsFrequencyInsideText(words, soup)


import requests
from bs4 import BeautifulSoup
import re

def extract_urls(url):
    try:
        # Make a GET request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad responses

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Modify the regular expression to only find links starting with https://www.oracle.com/
        # https://www\.oracle\.com/il-en/)[a-zA-Z0-9\-_/.]+(?<!\.svg|\.pdf|\.css|\.jpg|\.js
        pattern = r'https://www.oracle.com/il-en/[a-zA-Z0-9\-_./]+'
        urls = re.findall(pattern, str(soup))

        # Extract and return both URLs and text content
        return urls, soup.get_text()

    except Exception as e:
        # print(f"Error: {e}")
        return [], ''

import requests
from bs4 import BeautifulSoup
import re
from collections import Counter
from concurrent.futures import ThreadPoolExecutor, as_completed


# Assuming your existing functions are defined here: fetch_page, extract_urls, wordsFrequencyInsideText, search_for_words_in_url_optimized
def parallel_fetch_and_process(urls, words_to_find):
    url_mapped_to_words_frequencies = {}
    seen_urls = set()

    # Define a task for processing a single URL
    def process_url(url):
        if url not in seen_urls:
            seen_urls.add(url)
            return url, search_for_words_in_url_optimized(url, words_to_find)
        return url, None


      # Use ThreadPoolExecutor to fetch and process URLs in parallel
    with ThreadPoolExecutor(max_workers=10) as executor:
        # Submit tasks for all URLs
        future_to_url = {executor.submit(process_url, url): url for url in urls}

        # Process as tasks complete
        for future in as_completed(future_to_url):
            url = future_to_url[future]
            try:
                result = future.result()
                if result and result[1]:  # Check if there is a non-None result
                    url_mapped_to_words_frequencies[result[0]] = result[1]
            except Exception as exc:
                print(f'{url} generated an exception: {exc}')

    return url_mapped_to_words_frequencies

In [ ]:
#scans all of oracle site for the terms in words_to_find
#crawl is binary True/False -whether to crawl for more urls or not
#main_url : starting point of crawling
def scan_site_for_terms(words_to_find, crawl,main_url):
  main_url = main_url
  all_urls=[]
  if crawl:
    all_urls = extract_urls(main_url)[0]

  # Add the main URL to the list to be processed
  all_urls.insert(0, main_url)
  # Use the parallel fetch and process function
  url_mapped_to_words_frequencies = parallel_fetch_and_process(all_urls, words_to_find)
  word_to_urls_frequencies = {word: [] for word in words_to_find}

  for url, word_freqs in url_mapped_to_words_frequencies.items():
      # Iterate over each word and its frequency
      for word, freq in word_freqs.items():
        word_to_urls_frequencies[word].append((url, freq))

  for word, tuples in word_to_urls_frequencies.items():
    word_to_urls_frequencies[word] = sorted(tuples, key=lambda x: x[1], reverse=True)

  return word_to_urls_frequencies

In [ ]:

import IPython


import requests
from google.colab import output
from IPython.display import display, HTML

import requests
from IPython.display import display, HTML

def admin_screen():

  # Firebase Realtime Database URL
  firebase_url = 'https://razicloudlearn-default-rtdb.europe-west1.firebasedatabase.app/index.json'

  # Make a GET request to retrieve data from Firebase
  response = requests.get(firebase_url)

  # Check if the request was successful (status code 200)
  if response.status_code == 200:
      # Parse the JSON response
      result = response.json()

      # Extract index data
      index_data = {}
      for key, value in result.items():
          term = value['term']
          doc_ids = value['DocIDs']
          if term not in index_data:
              index_data[term] = []
          for doc_id in doc_ids:
              url = doc_id[0]
              frequency = doc_id[1]
              index_data[term].append({'url': url, 'frequency': frequency})




      # Generate HTML content for index display
      index_html_content = """

      """
      for term, data in index_data.items():
          index_html_content += f"""<h2 class="pt-4 text-bold text-xl">{term}</h2>"""
          index_html_content += """
          <div class="scrollable-table border-2">
              <table>
                  <tr>
                      """+f"""<th>URLs for {term}</th>"""+"""
                      <th>Frequency</th>
                      <th>Action</th>
                  </tr>
          """
          for item in data:
              index_html_content += f"""
                  <tr>
                      <td><a href="{item['url']}">{item['url']}</a></td>
                      <td>{item['frequency']}</td>
                      <td><button onclick="removeUrl('{term}', '{item['url']}')">Remove</button></td>
                  </tr>
              """
          index_html_content += """
              </table>
          </div>
          """

      index_html_content += """
      </body>
      </html>
      """

      # Generate HTML content for the entire page
      html_content = """
      <!DOCTYPE html>
      <html lang="en">
      <head>
          <meta charset="UTF-8">
          <meta name="viewport" content="width=device-width, initial-scale=1.0">
          <title>Manage Index - Admin Panel</title>
          <style>
              body {
                  font-family: Arial, sans-serif;
                  margin: 0;
                  padding: 20px;
              }
              h1 {
                  text-align: center;
              }
              table {
                  width: 100%;
                  border-collapse: collapse;
                  margin-top: 20px;
              }
              th, td {
                  border: 1px solid #ddd;
                  padding: 8px;
                  text-align: left;
              }
              th {
                  background-color: #f2f2f2;
              }
              tr:nth-child(even) {
                  background-color: #f2f2f2;
              }
              .scrollable-table {
                  max-height: 200px;
                  overflow-y: auto;
              }
          </style>
          <script src="https://cdn.tailwindcss.com"></script>
      </head>
      <body>


          <div class="border-b-4 border-[#007bff] mb-6 pt-2 flow-root">
              <div class="text-3xl text-orange-600 float-left">Oracle CloudSeeker</div>
              <button class="rounded-lg px-2 py-2 float-right" type="button" onclick="backToLogin()">Back to login</button>
              <button class="bg-[#007bff] text-[#fff] rounded-lg px-2 my-2 float-right" type="button" onclick="viewStats()">View Statistics</button>
          </div>
          <h1 class="text-3xl mb-2">Manage Index - Admin Panel</h1>
          <div class="shadow p-4">
              <form id="removeTermForm">
                  <label for="removeTermSelect">Select term to remove:</label>
                  <select class="bg-[#007bff] text-[#fff] rounded-lg px-2 py-2" id="removeTermSelect" name="removeTermSelect">
                      <option value="">Select a term to remove</option>
      """

      # Populate dropdown with existing terms
      for term in index_data.keys():
          html_content += f"<option value='{term}'>{term}</option>"

      html_content += """
                  </select>
                  <button class="bg-[#007bff] text-[#fff] rounded-lg px-2 py-2" type="button" onclick="removeTerm()">Remove Term Entirely from Index</button>
              </form>
          </div>

          <div class="shadow p-4">
              <form id="addUrlForm">
                  <label for="existingTerm">Select existing term:</label>
                  <select class="bg-[#007bff] text-[#fff] rounded-lg px-2 py-2" id="existingTerm" name="existingTerm">
      """

      # Populate dropdown with existing terms
      for term in index_data.keys():
          html_content += f"<option value='{term}'>{term}</option>"

      html_content += """
                  </select>
                  <label for="newUrl">New URL:</label>
                  <input class="rounded-lg py-2" type="text" id="newUrl" name="newUrl">
                  <button class="bg-[#007bff] text-[#fff] rounded-lg px-2 py-2" type="button" onclick="addUrl()">Add URL</button>
              </form>
          </div>

          <div class="shadow p-4">
              <form id="scanSiteForm">
                  <label for="newTerm">Enter new term:</label>
                  <input class="rounded-lg py-2" type="text" id="newTerm" name="newTerm">
                  <button class="bg-[#007bff] text-[#fff] rounded-lg px-2 py-2" type="button" onclick="scanSite()">Scan Site for This Term</button>
              </form>
          </div>

      """

      # Append the index display HTML
      html_content += index_html_content

      # Add JavaScript functions
      html_content += """
          <script>
              function backToLogin() {
                  // Implement your logic here
              }

              function viewStats() {
                  // Implement your logic here
              }

              function removeTerm() {
                  var term = document.getElementById('newTerm').value;
                  console.log(term);
                  if(!term){
                    return;
                  }
                  google.colab.kernel.invokeFunction('notebook.remove_term', [term], {});
                  showAlert(`The Term ${term} will be removed in a couple of seconds`);
                  term.value = '';

              }

              function removeUrl(term, url) {
                  // Confirm if the user wants to remove the URL
                  if (confirm(`Are you sure you want to remove the URL '${url}' for term '${term}'?`)) {
                      // Make a request to remove the URL from Firebase
                      google.colab.kernel.invokeFunction('notebook.remove_url', [term, url], {});
                  }
              }

              function addUrl() {
                  var term = document.getElementById('existingTerm').value;
                  var url = document.getElementById('newUrl').value;

                  // Check if the URL starts with "https://www.oracle.com/il-en/"
                  if (!url.startsWith("https://www.oracle.com/il-en/")) {
                      showAlert("URL must start with 'https://www.oracle.com/il-en/'.");
                      return;
                  }



                  // Check if the URL is valid (page found)
                  fetch(url)
                      .then(response => {
                          if (!response.ok) {
                              showAlert("Invalid URL. Page not found.");
                          } else {
                              // Make a request to add the new URL for the existing term
                              google.colab.kernel.invokeFunction('notebook.add_url', [term, url,0,True], {});
                              showAlert("URL added successfully.");
                              // Reload the page to refresh the table
                              //location.reload();

                          }
                      })
                      .catch(error => {
                          showAlert("Error checking URL validity.");
                      });

              }


              function scanSite() {
                  var newTerm = document.getElementById('newTerm').value;
                  if(!newTerm){
                    return;
                  }
                  // Make a request to scan the site for the new term
                  google.colab.kernel.invokeFunction('notebook.scan_site_for_term', [newTerm], {});
                  showAlert("Site was scanned for chosen term. Please Wait A couple of seconds to update");
                  //location.reload();
              }

              function showAlert(message) {
                  alert(message);
              }
          </script>
      </body>
      </html>
      """

      # Output HTML content
      #display(IPython.display.HTML(html_content))
      #display(HTML(html_content))
      return html_content
  else:
      print(f"Failed to retrieve data from Firebase: {response.status_code}")




############# Implement the Functions ########


#findFreq is boolean -whether to find frequency for url or not
def add_url(term, url,frequency,findFreq):
    import requests
    import json  # Import the json module here
    from google.colab import output
    # Firebase Realtime Database URL
    result=0
    freq=frequency
    if findFreq:
      result=scan_site_for_terms([term],False,url)
      freq=result[term][0][1]

    firebase_url = 'https://razicloudlearn-default-rtdb.europe-west1.firebasedatabase.app/index.json'

    # Make a GET request to retrieve data from Firebase
    response = requests.get(firebase_url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON response
        result = response.json()

        # Update the data to include the new URL for the specified term
        for key, value in result.items():
            if value['term'] == term:
                value['DocIDs'].append([url, freq])
                # Make a PUT request to update the data in Firebase
                response = requests.put(firebase_url, json=result)
                if response.status_code == 200:
                    success_message = f"Successfully added URL '{url}' for term '{term}'"
                    json.dumps(success_message)
                else:
                    error_message = f"Failed to add URL '{url}' for term '{term}': {response.status_code}"
                    output.eval_js(f"alert('{error_message}')")
                break
    else:
        error_message = f"Failed to retrieve data from Firebase: {response.status_code}"
        output.eval_js(f"alert('{error_message}')")
    clear_output(wait=True)
    present_admin_screen()

# Register the function to be invoked from JavaScript
output.register_callback('notebook.add_url', add_url)



#######################remove url##############
def remove_url(term, url):
    import requests

    # Firebase Realtime Database URL
    firebase_url = 'https://razicloudlearn-default-rtdb.europe-west1.firebasedatabase.app/index.json'

    # Make a GET request to retrieve data from Firebase
    response = requests.get(firebase_url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON response
        result = response.json()

        # Remove the specified URL for the given term
        for key, value in result.items():
            if value['term'] == term:
                doc_ids = value['DocIDs']
                for i, doc_id in enumerate(doc_ids):
                    if doc_id[0] == url:
                        del result[key]['DocIDs'][i]
                        break

                # Make a PUT request to update the data in Firebase
                response = requests.put(firebase_url, json=result)

                if response.status_code == 200:
                    return f"Successfully removed URL '{url}' for term '{term}'"
                else:
                    return f"Failed to remove URL '{url}' for term '{term}': {response.status_code}"
    else:
        return f"Failed to retrieve data from Firebase: {response.status_code}"

# Register the function to be invoked from JavaScript
output.register_callback('notebook.remove_url', remove_url)



###################scan site for term
def scan_site_for_term(term):

  term= term.lower()
  url="https://www.oracle.com/il-en/"
  result = scan_site_for_terms([term],True,url)
  add_terms_urls_to_db(result)
  clear_output(wait=True)
  present_admin_screen()

# Register the function to be invoked from JavaScript
output.register_callback('notebook.scan_site_for_term', scan_site_for_term)


##########################add_terms urls_to db

from firebase import firebase
FBconn = firebase.FirebaseApplication(DB_URL,None)

def add_terms_urls_to_db(word_to_urls_frequencies):
  for word, tuples in word_to_urls_frequencies.items():
    data_to_upload = {
        'term' : word,
        'DocIDs': tuples
    }
    result = FBconn.post('./index/',data_to_upload)


############## Remove Term ###############

def remove_term(term):
  if not term:
    return
  index_path = './index'
  # Get all objects in 'index'
  index_objects = FBconn.get(index_path, None)
  if not index_objects:
      return
  for obj_id, obj_data in index_objects.items():
        if obj_data.get('term') == term:
            # Delete the object that contains the term
            FBconn.delete(index_path, obj_id)
            break
  clear_output(wait=True)
  present_admin_screen()

##########################
output.register_callback('notebook.remove_term', remove_term)



# **Chatbot Implementaion**

In [ ]:
import requests
import re
from nltk.chat.util import Chat, reflections
from bs4 import BeautifulSoup
import json

In [ ]:
def find_first_h2_or_h1_pattern(url):
    if not url:
      return "None", "None"

    response = requests.get(url)
    if response.status_code != 200:
      return "None", "None"

    soup = BeautifulSoup(response.text, 'html.parser')
    # First, try to find a <h2> followed by a <p>
    h1 = soup.find("h1")
    p = soup.find("p")
    # h2_p_pattern = soup.find(lambda tag: tag.name == "h2" and tag.next_sibling.name == "p")

    # If found, return the texts inside them
    if h1 and p:
        return h1.text, p.text

    # If neither pattern is found, return a message indicating so
    return "None", "None"




def getAllDataFromDB(db_url):
  url = f"{db_url}/index.json"
  response = requests.get(url)
  if response.status_code != 200:
    return []

  all_data = response.json()
  termsMappedToUrlFrequency = dict()
  for unique_key in all_data:
    term = all_data[unique_key]['term']
    if 'DocIDs' not in all_data[unique_key]:
      termsMappedToUrlFrequency[term] = ["None",0]
    termsMappedToUrlFrequency[term] = all_data[unique_key]['DocIDs'][:3]

  return termsMappedToUrlFrequency


# Function to create patterns based on the fetched data
def create_patterns_from_data(db_url):
    data = getAllDataFromDB(db_url)
    patterns = [
        # Common patterns
        (r'hi|hello|hey', ['Hello!', 'Hi there!', 'Hey!']),
        (r'how are you?', ['I\'m good, thank you!', 'I\'m doing well, thanks for asking.']),
        (r'what is your name?', ['You can call Zebra ChatBot.', 'I go by the name Zebra ChatBot.']),
        (r'your name (.*)', ['You can call me Zebra ChatBot.', 'I go by the name Zebra ChatBot.']),
        (r"(goodbye|bye|see ya)", ["Goodbye", "See you later",]),
        (r"sorry (.*)", ["It's alright", "It's OK, never mind",]),
        (r"(where|how) can i search|where is (the )?search", ['its basically above me', 'look above...', 'IT IS ABOVE YOU']),
        (r"(where can i find (my )?bookmarks?|where (is|are) (the )?bookmarks?)", ['Above the search result there is an bookmarks button click it', 'above the search click on bookmarks']),
        (r"(where can i find (my )?histor(y|ies)|where (is|are) (the )?history(y|ies))", ['Above the search result there is an history button click it', 'above the search click on history']),
        (r"where is the admin screen\??", ["Cant tell you, because you are a regular user"])
    ]

    if data:
        common_index_prefixes = ["what is ", "tell me more about ", "show me ", " ", "what is the meaning of ", "explain ", "what do you mean by ", "define ",
                                 "summarize ", "show me more about ", "", "what do you mean by ", "what do you mean ", "talk more about ", "give me link for ", "help me find "]
        for term, details in data.items():
          h1_text_array = []
          p_text_array = []
          for i in range(3):
            h1_text, p_text = find_first_h2_or_h1_pattern(details[i][0])
            h1_text_array.append(h1_text)
            p_text_array.append(p_text)

          for common_prefix in common_index_prefixes:
            answers = []
            for i in range(3):
              answers.append(json.dumps({'url':details[i][0], 'message_for_url':f"Check out these resources for more information on {term}",
                                                       'header_message':h1_text_array[i], 'paragraph_message':p_text_array[i]}))
            pattern = (rf'{common_prefix}{term}\s?\??', answers)
            patterns.append(pattern)

    return patterns
    # f"Check out these resources for more information on {term}: {details[0][0]}"
    # f"Check out these resources for more information on {term}: {details[2][0]}"

statistic


In [ ]:
def showStatics():
  import requests
  from collections import defaultdict
  import IPython
  from IPython.display import display, HTML, clear_output
  from google.colab import output

  # Firebase Realtime Database URL

  firebase_url = 'https://razicloudlearn-default-rtdb.europe-west1.firebasedatabase.app/index.json'

  # Make a GET request to retrieve data from Firebase
  response = requests.get(firebase_url)
  def go_to_Admin():
    # Clear the current output and display the login screen
    clear_output(wait=True)
    present_admin_screen()
  output.register_callback('notebook.go_to_Admin', go_to_Admin)

  # Check if the request was successful (status code 200)
  if response.status_code == 200:
      # Parse the JSON response
      result = response.json()

      # Extract index data
      index_data = []
      term_count = defaultdict(int)  # To count occurrences of each term
      for key, value in result.items():
          term = value['term']
          doc_ids = value['DocIDs']
          for doc_id in doc_ids:
              url = doc_id[0]
              frequency = doc_id[1]
              index_data.append({'term': term, 'url': url, 'frequency': frequency})
              term_count[term] += frequency

      # Generate HTML content
      html_content = """
      <!DOCTYPE html>
      <html lang="en">
      <head>
          <meta charset="UTF-8">
          <meta name="viewport" content="width=device-width, initial-scale=1.0">
          <title>Number of times term is mentioned per URL</title>
          <style>
              body {
                  font-family: Arial, sans-serif;
                  margin: 0;
                  padding: 20px;
              }
              h1 {
                  text-align: center;
              }
              table {
                  width: 100%;
                  border-collapse: collapse;
                  margin-top: 20px;
              }
              th, td {
                  border: 1px solid #ddd;
                  padding: 8px;
                  text-align: left;
              }
              th {
                  background-color: #f2f2f2;
              }
              tr:nth-child(even) {
                  background-color: #f2f2f2;
              }
              .chart-container {
                  width: 80%;
                  margin: auto;
                  margin-top: 20px;
              }
              .button-container {
                  text-align: center;
                  margin-top: 20px;
              }
              .button {
                  margin: 0 10px;
                  padding: 10px 20px;
                  font-size: 16px;
                  cursor: pointer;
                  background-color: #007bff;
                  color: #fff;
                  border: none;
                  border-radius: 5px;
              }
              .buttonback {
                  margin: 0 10px;
                  padding: 10px 20px;
                  font-size: 16px;
                  cursor: pointer;
                  background-color: gray;
                  color: #fff;
                  border: none;
                  border-radius: 5px;
              }
              .button:hover {
                  background-color: #0056b3;
              }
          </style>
      </head>
      <body>
          <h1 id="title">Statistics</h1>

          <div class="button-container">
              <button class="buttonback" id="backtoadmin" onclick="backToAdmin()">Back To Admin Panel</button>
              <script>
                function backToAdmin(){
                      google.colab.kernel.invokeFunction('notebook.go_to_Admin', [], {});


                }
              </script>
              <button class="button" id="totalButton" onclick="showTotalStatistics()">Total count in the website</button>
              <button class="button" id="termWebsiteButton" onclick="showTermWebsiteStatistics()">Term in the website</button>

          </div>

          <div id="chartSection" style="display: none;">
              <label for="termSelector">Select a term:</label>
              <select id="termSelector" onchange="showStatistics()">
                  <option value="" selected disabled>Select a term</option>
          """

      # Add options for term selector
      for term, count in term_count.items():
          html_content += f'<option value="{term}">{term}</option>'

      html_content += """
              </select>

              <div class="chart-container">
                  <canvas id="barChart"></canvas>
              </div>
          </div>

          <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
          <script>
              var myChart; // Declare myChart globally
              var indexData = """ + str(index_data) + """;
              var termCount = """ + str(dict(term_count)) + """;

              function showTotalStatistics() {
                  document.getElementById("chartSection").style.display = "block";
                  document.getElementById("title").innerText = "Total count in the website";
                  document.getElementById("termSelector").disabled = true;
                  document.getElementById("termSelector").innerHTML = "<option value='' selected disabled>Select a term</option>";
                  var labels = [];
                  var data = [];
                  Object.keys(termCount).sort(function(a, b) {
                      return termCount[b] - termCount[a];
                  }).forEach(function(term) {
                      labels.push(term);
                      data.push(termCount[term]);
                  });
                  displayChart(labels, data);
              }

              function showTermWebsiteStatistics() {
                  document.getElementById("chartSection").style.display = "block";
                  document.getElementById("title").innerText = "Term in the website";
                  document.getElementById("termSelector").disabled = false;
                  document.getElementById("termSelector").innerHTML = "<option value='' selected disabled>Select a term</option>";

                  // Add options for term selector with count of each term in the website
                  Object.keys(termCount).forEach(function(term) {
                      var count = 0;
                      indexData.forEach(function(item) {
                          if (item.term === term) {
                              count += item.frequency;
                          }
                      });
                      document.getElementById("termSelector").innerHTML += "<option value='" + term + "'>" + term + " (" + count + ")</option>";
                  });

                  clearChart();
              }

              function showStatistics() {
                  var selectedTerm = document.getElementById('termSelector').value;
                  var selectedOption = document.getElementById('termSelector').options[document.getElementById('termSelector').selectedIndex].text;

                  if (selectedOption === "Term in the website") {
                      showTotalStatistics();
                  } else {
                      var filteredData = indexData.filter(function(item) {
                          return item.term === selectedTerm;
                      });
                      var labels = filteredData.map(function(item) {
                          return item.url;
                      });
                      var data = filteredData.map(function(item) {
                          return item.frequency;
                      });
                      displayChart(labels, data);
                  }
              }

              function displayChart(labels, data) {
                  // Destroy the previous chart if exists
                  clearChart();

                  // Create a new chart
                  var ctx = document.getElementById('barChart').getContext('2d');
                  myChart = new Chart(ctx, {
                      type: 'bar',
                      data: {
                          labels: labels,
                          datasets: [{
                              label: 'Frequency',
                              backgroundColor: 'rgba(54, 162, 235, 0.5)',
                              borderColor: 'rgba(54, 162, 235, 1)',
                              borderWidth: 1,
                              data: data
                          }]
                      },
                      options: {
                          scales: {
                              yAxes: [{
                                  ticks: {
                                      beginAtZero: true
                                  }
                              }]
                          }
                      }
                  });
              }

              function clearChart() {
                  // Clear the chart
                  if (myChart) {
                      myChart.destroy();
                  }
              }
          </script>
      </body>
      </html>
      """

      # Output HTML content
      display(HTML(html_content))
  else:
      print(f"Failed to retrieve data from Firebase: {response.status_code}")



In [ ]:
#---------------------------------------------------------------admin_screen--------------------------------------------------------------------------------------------
def present_admin_screen():
  import requests
  from IPython.display import display, HTML

  # Firebase Realtime Database URL
  firebase_url = 'https://razicloudlearn-default-rtdb.europe-west1.firebasedatabase.app/index.json'

  # Make a GET request to retrieve data from Firebase
  response = requests.get(firebase_url)

  # Check if the request was successful (status code 200)
  if response.status_code == 200:
      # Parse the JSON response
      result = response.json()

      # Extract index data
      index_data = {}
      for key, value in result.items():
          term = value['term']
          doc_ids = value['DocIDs']
          if term not in index_data:
              index_data[term] = []
          for doc_id in doc_ids:
              url = doc_id[0]
              frequency = doc_id[1]
              index_data[term].append({'url': url, 'frequency': frequency})

      # sort each array by its frequency
      for term, docs in index_data.items():
        index_data[term] = sorted(docs, key=lambda x: x['frequency'], reverse=True)

      # Generate HTML content for index display
      index_html_content = """

      """
      for term, data in index_data.items():
          index_html_content += f"""<h2 class="pt-4 text-bold text-xl">{term}</h2>"""
          index_html_content += """
          <div class="scrollable-table border-2">
              <table>
                  <tr>
                      """+f"""<th>URLs for {term}</th>"""+"""
                      <th>Frequency</th>
                      <th>Action</th>
                  </tr>
          """
          for item in data:
              index_html_content += f"""
                  <tr>
                      <td><a href="{item['url']}">{item['url']}</a></td>
                      <td>{item['frequency']}</td>
                      <td><button onclick="removeUrl('{term}', '{item['url']}')">Remove</button></td>
                  </tr>
              """
          index_html_content += """
              </table>
          </div>
          """

      index_html_content += """
      </body>
      </html>
      """

      # Generate HTML content for the entire page
      html_content = """
      <!DOCTYPE html>
      <html lang="en">
      <head>
          <meta charset="UTF-8">
          <meta name="viewport" content="width=device-width, initial-scale=1.0">
          <title>Manage Index - Admin Panel</title>
          <style>
              body {
                  font-family: Arial, sans-serif;
                  margin: 0;
                  padding: 20px;
              }
              h1 {
                  text-align: center;
              }
              table {
                  width: 100%;
                  border-collapse: collapse;
                  margin-top: 20px;
              }
              th, td {
                  border: 1px solid #ddd;
                  padding: 8px;
                  text-align: left;
              }
              th {
                  background-color: #f2f2f2;
              }
              tr:nth-child(even) {
                  background-color: #f2f2f2;
              }
              .scrollable-table {
                  max-height: 200px;
                  overflow-y: auto;
              }
          </style>
          <script src="https://cdn.tailwindcss.com"></script>
      </head>
      <body>


          <div class="border-b-4 border-[#007bff] mb-6 pt-2 flow-root">
              <div class="text-3xl text-orange-600 float-left">Oracle CloudSeeker</div>
              <button class="rounded-lg px-2 py-2 float-right" type="button" onclick="backToLogin()">Back to login</button>
              <button class="bg-[#007bff] text-[#fff] rounded-lg px-2 my-2 float-right" type="button" onclick="viewStats()">View Statistics</button>
              <script>
                  function viewStats(){
                      google.colab.kernel.invokeFunction('notebook.go_to_stats', [], {});

                  }
              </script>
          </div>
          <h1 class="text-3xl mb-2">Manage Index - Admin Panel</h1>
          <div class="shadow p-4">
              <form id="removeTermForm">
                  <label for="removeTermSelect">Select term to remove:</label>
                  <select class="bg-[#007bff] text-[#fff] rounded-lg px-2 py-2" id="removeTermSelect" name="removeTermSelect">
                      <option value="">Select a term to remove</option>
      """
      def go_to_stats():
          # Clear the current output and display the login screen
          clear_output(wait=True)
          showStatics()

      # Register the function to be invoked from JavaScript
      output.register_callback('notebook.go_to_stats', go_to_stats)
      # Populate dropdown with existing terms
      for term in index_data.keys():
          html_content += f"<option value='{term}'>{term}</option>"

      html_content += """
                  </select>
                  <button class="bg-[#007bff] text-[#fff] rounded-lg px-2 py-2" type="button" onclick="removeTerm()">Remove Term Entirely from Index</button>
              </form>
          </div>

          <div class="shadow p-4">
              <form id="addUrlForm">
                  <label for="existingTerm">Select existing term:</label>
                  <select class="bg-[#007bff] text-[#fff] rounded-lg px-2 py-2" id="existingTerm" name="existingTerm">
      """

      # Populate dropdown with existing terms
      for term in index_data.keys():
          html_content += f"<option value='{term}'>{term}</option>"

      html_content += """
                  </select>
                  <label for="newUrl">New URL:</label>
                  <input class="rounded-lg py-2" type="text" id="newUrl" name="newUrl">
                  <button class="bg-[#007bff] text-[#fff] rounded-lg px-2 py-2" type="button" onclick="addUrl()">Add URL</button>
              </form>
          </div>

          <div class="shadow p-4">
              <form id="scanSiteForm">
                  <label for="newTerm">Enter new term:</label>
                  <input class="rounded-lg py-2" type="text" id="newTerm" name="newTerm">
                  <button class="bg-[#007bff] text-[#fff] rounded-lg px-2 py-2" type="button" onclick="scanSite()">Scan Site for This Term</button>
              </form>
          </div>

      """

      # Append the index display HTML
      html_content += index_html_content

      # Add JavaScript functions
      html_content += """
          <script>
              function backToLogin() {
                  event.preventDefault();
                  google.colab.kernel.invokeFunction('notebook.go_to_logout', [], {});
              }

              function viewStats() {
                 event.preventDefault();
                 google.colab.kernel.invokeFunction('notebook.go_to_stats', [], {});
                            }

              function removeTerm() {
                  var term = document.getElementById('removeTermSelect').value;
                  console.log(term);
                  if(!term){
                    return;
                  }
                  google.colab.kernel.invokeFunction('notebook.remove_term', [term], {});
                  showAlert(`The Term ${term} will be removed in a couple of seconds`);
                  document.getElementById('newTerm').value = '';
              }

              function removeUrl(term, url) {
                  // Confirm if the user wants to remove the URL
                  if (confirm(`Are you sure you want to remove the URL '${url}' for term '${term}'?`)) {
                      // Make a request to remove the URL from Firebase
                      google.colab.kernel.invokeFunction('notebook.remove_url', [term, url], {});
                  }
              }

              function addUrl() {
                  var term = document.getElementById('existingTerm').value;
                  var url = document.getElementById('newUrl').value;

                  // Check if the URL starts with "https://www.oracle.com/il-en/"
                  if (!url.startsWith("https://www.oracle.com/il-en/")) {
                      showAlert("URL must start with 'https://www.oracle.com/il-en/'.");
                      return;
                  }
                  // Make a request to add the new URL for the existing term
                  google.colab.kernel.invokeFunction('notebook.add_url', [term, url,0,true], {});
                  showAlert("Trying to add URL.");
              }


              function scanSite() {
                  var newTerm = document.getElementById('newTerm').value;

                  // Make a request to scan the site for the new term
                  google.colab.kernel.invokeFunction('notebook.scan_site_for_term', [newTerm], {});
                  showAlert("We are currently Scanning for the new Term. The site will refresh once done");
                  //location.reload();
              }

              function showAlert(message) {
                  alert(message);
              }
          </script>
      </body>
      </html>
      """

      # Output HTML content
      display(IPython.display.HTML(html_content))
      #display(HTML(html_content))
  else:
      print(f"Failed to retrieve data from Firebase: {response.status_code}")


In [ ]:
import IPython
from IPython.display import display, HTML, clear_output, Javascript
from google.colab import output
from datetime import datetime





#--------------------------------Register Screen----------------------------------
flag=False

def check_username_existence(username):
    # Initialize Firebase connection
    FBconn = firebase.FirebaseApplication('https://razicloudlearn-default-rtdb.europe-west1.firebasedatabase.app/', None)

    # Make a GET request to retrieve all users
    response = FBconn.get('/users', None)

    # Check if the response contains data
    if response:
        # Iterate through the users to check if the username exists
        for key, value in response.items():
            if value['username'] == username:
                return True
    return False

def register_user(username, password):
    # Check if the username already exists
    if check_username_existence(username):
        # Display an error message
        display_error_message("Username already exists. Please choose a different username.", True)
    else:
        # Initialize Firebase connection
        FBconn = firebase.FirebaseApplication('https://razicloudlearn-default-rtdb.europe-west1.firebasedatabase.app/', None)

        # Prepare data to be uploaded
        data_to_upload = {
            'username': username,
            'password': password,
            'history': ['history-placeholder'],
            'bookmarks': ['bookmarks-placeholder']
        }

        # Post the new user data to Firebase
        result = FBconn.post('/users', data_to_upload)

        # Hide the error message
        display_error_message("", False)

def display_error_message(message, show):
    global flag
    # Create an HTML element for the error message
    error_message_html = f'<p id="errorMessage" class="text-red-500 text-sm italic">{message}</p>'

    # Hide or show the error message based on the 'show' parameter
    if not flag:
      display(IPython.display.HTML(error_message_html))
    elif show:
        display(IPython.display.HTML("""<script>document.getElementById('errorMessage').style.display = 'block';</script>"""))
    else:
        display(IPython.display.HTML("""<script>document.getElementById('errorMessage').style.display = 'none';</script>"""))

    flag = True

# Register the function to be invoked from JavaScript
output.register_callback('notebook.register_user', register_user)




#--------------------------------Login Screen-------------------------------------
import requests
from google.colab import output
from firebase import firebase
import IPython

def go_to_login():
    # Clear the current output and display the login screen
    clear_output(wait=True)
    display(IPython.display.HTML(login_screen_html))

# Register the function to be invoked from JavaScript
output.register_callback('notebook.go_to_login', go_to_login)


def go_to_register():
    # Clear the current output and display the register screen
    clear_output(wait=True)
    display(IPython.display.HTML(register_screen_html))

# Register the function to be invoked from JavaScript
output.register_callback('notebook.go_to_register', go_to_register)


def go_to_search():
    # Clear the current output and display the login screen
    clear_output(wait=True)
    display(IPython.display.HTML(search_bar_html))

# Register the function to be invoked from JavaScript
output.register_callback('notebook.go_to_search', go_to_search)

def go_to_admin():
    # clear_output(wait=True)
    admin_screen()
    # display(IPython.display.HTML(admin_screen()))

output.register_callback('notebook.go_to_admin', go_to_admin)
#---------------
# HTML code for the search bar screen
#---------------
flag = False
def check_user_credentials(username, password):
    # Initialize Firebase connection
    FBconn = firebase.FirebaseApplication('https://razicloudlearn-default-rtdb.europe-west1.firebasedatabase.app/', None)

    # Make a GET request to retrieve all users
    response = FBconn.get('/users', None)

    # Check if the response contains data
    if response:
        # Iterate through the users to check if the username and password match
        for key, value in response.items():
            if value['username'] == username and value['password'] == password:
                return True
    return False

def login_user(username, password):
    global valid_username, valid_password
    print(f"Attempting login for username: {username}")
    if check_user_credentials(username, password):
        if (username=='admin'):
          valid_username = username
          valid_password = password
          clear_output(wait=True)
          present_admin_screen()
        else:
          valid_username = username
          valid_password = password
          clear_output(wait=True)
          display(IPython.display.HTML(search_bar_html))
          display(IPython.display.HTML(search_results_html))
          display(IPython.display.HTML(htmlContentForChatBot))
        #display_message("Valid user", "text-green-500")  # Display success message in green
    else:
        display_message("Incorrect user details", "text-red-500")  # Display error message in red

def display_message(message, text_color):
    global flag
    # Create an HTML element for the message
    message_html = f'<p id="loginMessage" class="text-sm italic {text_color}">{message}</p>'

    # Hide or show the message based on the 'show' parameter
    if not flag:
        display(IPython.display.HTML(message_html))
    elif text_color == "text-green-500":
        display(IPython.display.HTML("""<script>document.getElementById('loginMessage').style.color = 'green';</script>"""))
    else:
        display(IPython.display.HTML("""<script>document.getElementById('loginMessage').style.color = 'red';</script>"""))

    flag = True
# Register the function to be invoked from JavaScript
output.register_callback('notebook.login_user', login_user)
# Display the login screen with Tailwind CSS
display(IPython.display.HTML(login_screen_html))


#-------------------------------Search Screen-------------------------------------
DB_URL = "https://razicloudlearn-default-rtdb.europe-west1.firebasedatabase.app/"

#--------------screen navigation in search----------------
def go_to_logout():
    # Clear the current output and display the login screen
    clear_output(wait=True)
    valid_username = None
    display(IPython.display.HTML(login_screen_html))

output.register_callback('notebook.go_to_logout', go_to_logout)

def go_to_bookmark():
    clear_output(wait=True)
    display(HTML(bookmarks_page()))

output.register_callback('notebook.go_to_bookmark', go_to_bookmark)

def go_to_history_page():
    clear_output(wait=True)
    display(HTML(history_page()))

output.register_callback('notebook.go_to_history_page', go_to_history_page)


# def go_to_history():
#     # Clear the current output and display the login screen
#     clear_output(wait=True)
#     display(HTML(history_page_html))


# output.register_callback('notebook.go_to_history', go_to_history)


# Display the search bar screen initially
#display(IPython.display.HTML(search_bar_html))
#display(IPython.display.HTML(search_results_html))
def update_user_history_by_username(username, query):
    # Construct the URL to fetch the users' data from Firebase
    users_url = f"{DB_URL}/users.json"

    # Fetch the users' data from Firebase
    response = requests.get(users_url)
    if response.ok:
        users_data = response.json()

        # Find the user by username
        for user_id, user_data in users_data.items():
            if user_data["username"] == username:
                current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

                # Update the user's history
                user_data["history"].append({"query": query, "datetime": current_datetime})

                # Update the user's data in Firebase
                update_user_url = f"{DB_URL}/users/{user_id}.json"
                update_response = requests.put(update_user_url, json=user_data)
                break  # Stop searching once user is found and history is updated

        else:
            print("Username not found in database.")
    else:
        print("Failed to fetch users' data from database.")

def savetobookmarks(url):
    try:
# https://razicloudlearn-default-rtdb.europe-west1.firebasedatabase.app/

        # Initialize Firebase connection
        FBconn = firebase.FirebaseApplication('https://razicloudlearn-default-rtdb.europe-west1.firebasedatabase.app/', None)

        # Fetch the users collection
        users_data = FBconn.get('/users', None)

        # Find the user with the matching username (valid_username)
        user_id = None
        for uid, user_data in users_data.items():
            if user_data.get('username') == valid_username:
                user_id = uid
                break

        if user_id:
            # Fetch the current bookmarks for the user
            user_data = FBconn.get(f'/users/{user_id}', None)
            current_bookmarks = user_data.get('bookmarks', [])

            # Check if the URL already exists in the bookmarks
            if url not in current_bookmarks:
                # Add the new bookmark to the list
                current_bookmarks.append(url)

                # Update the user's bookmarks in Firebase
                FBconn.put(f'/users/{user_id}', 'bookmarks', current_bookmarks)

                # Confirm the bookmark has been saved
                # Display an alert
                alert_message = f"Bookmark saved for {url}"
                display(IPython.display.Javascript(f'alert("{alert_message}")'))
            else:
                alert_message = f"URL {url} already exists in bookmarks."
                display(IPython.display.Javascript(f'alert("{alert_message}")'))

    except Exception as e:
        print(f"Error: {e}")

def get_user_bookmarks_by_username(username):
    # Construct the URL to fetch the users' data from Firebase
    users_url = f"{DB_URL}/users.json"

    # Fetch the users' data from Firebase
    response = requests.get(users_url)
    if response.ok:
        users_data = response.json()

        # Find the user by username
        for user_id, user_data in users_data.items():
            if user_data["username"] == username:
                # Return the user's history
                return user_data.get("bookmarks", [])

        else:
            print("Username not found in database.")
            return []

    else:
        print("Failed to fetch users' data from the database.")
        return []




output.register_callback('notebook.savetobookmarks', savetobookmarks)

def get_user_bookmarks_by_username(username):
    # Initialize Firebase connection
    FBconn = firebase.FirebaseApplication('https://razicloudlearn-default-rtdb.europe-west1.firebasedatabase.app/', None)

    # Fetch the user's data
    user_data = FBconn.get('/users', None)

    # Find the user by username
    for user_id, data in user_data.items():
        if data.get('username') == username:
            # Return the user's bookmarks
            return data.get('bookmarks', [])

    print(f"Username {username} not found.")
    return []

def bookmarks_page():
    # Fetch user bookmarks using the existing function or logic
    user_bookmarks = get_user_bookmarks_by_username(valid_username)

    # Process the bookmarks
    if user_bookmarks:
        # Exclude the first bookmark
        bookmarks_except_first = user_bookmarks[1:]

        # Generate HTML for the bookmarks page
        html_code = f"""
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Bookmarks Page</title>
            <style>
                body {{
                    font-family: Arial, sans-serif;
                    margin: 20px;
                }}

                header {{
                    text-align: center;
                    padding: 10px;
                    background-color: #f2f2f2;
                }}

                section {{
                    margin-top: 20px;
                }}

                .bookmark {{
                    border: 1px solid #ccc;
                    padding: 10px;
                    margin-bottom: 10px;
                }}

                footer {{
                    text-align: center;
                    padding: 10px;
                    background-color: #f2f2f2;
                }}
            </style>
        </head>
        <body>
            <header>
                <a href="#" onclick="goBack(event)" class="back-icon">
                    <img src="https://img.icons8.com/ios/452/circled-left-2.png" alt="Back" width="30" height="30">
                </a>
                <h1>Bookmarks Page</h1>
            </header>

            <section>
                {"".join(f'<div class="bookmark"><p><strong>Bookmark URL:</strong> {bookmark}</p></div>' for bookmark in bookmarks_except_first)}
            </section>

            <footer>
                <p>&copy; 2024 Bookmarks Page. All rights reserved.</p>
            </footer>
            <script>
                function goBack(event) {{
                    event.preventDefault();
                    google.colab.kernel.invokeFunction('notebook.go_to_search', [], {{}});
                }}
            </script>
        </body>
        </html>
        """
        return html_code
    else:
        return "No bookmarks found."




def search(query):
    # Dummy search results for demonstration
    # display(IPython.display.Javascript("clearSearchResults()"))
    query = re.sub(r'[^a-zA-Z ]', '', query)
    linkes_rankes = searchAndRankLinks(query, DB_URL)
    linkes_rankes = linkes_rankes[:5]

    # Generate HTML for search results
    results_html = ""
    for result in linkes_rankes:
        url = result[0]
        rank = result[1]
        results_html += f"""<div class="result shadow px-4 py-4 flex gap-4 items-center">
                              <a href="javascript:void(0);" onclick="openInNewTab('{url}')">{url}</a>
                              <p>Rank: {rank}</p>
                              <button class="text-3xl float-right" onclick="saveBookmark('{url}')">&#9734;</button> <!--empty star -->
                            </div>"""

    # Display the search results screen with the generated HTML

    js_code = f'''
    updateSearchResults(`{results_html}`)
    '''
    display(IPython.display.Javascript(js_code))
    # display(IPython.display.HTML(results_html))
    update_user_history_by_username(valid_username, query)
    # JavaScript function to open a URL in a new tab/window and focus on the original window
    display(IPython.display.Javascript('''
        function openInNewTab(url) {
            var win = window.open(url, '_blank');
            win.focus();
        }
        function saveBookmark(url) {
          google.colab.kernel.invokeFunction('notebook.savetobookmarks', [url], {});
        }

    '''))

    output.eval_js(js_code)

# Register the search function to be called from JavaScript
output.register_callback('notebook.search', search)

#--------------------------------I Feel Lucky Implementaion ----------------------------------
import random
def lucky():
  interesting_terms = [
    "Docker IaaS Container",
    "Cloud Python Scalability",
    "Docker PaaS Flexibility",
    "Python PaaS Container",
    "API Flexibility",
    "Computing Service",
    "Container Technology Scalability",
    "SaaS Python Computing",
    "SaaS Docker Oracle",
    "Python IaaS Flexibility",
    "PaaS Container",
    "SaaS API Technology",
    "Computing Technology",
    "Docker Python Scalability",
    "PaaS IaaS Scalability",
    "Python Technology Application",
    "Container Service Application",
    "Cloud Container",
    "Docker Technology Oracle",
    "Provider Application",
    "database"
  ]
  random_term = random.choice(interesting_terms)
  search(random_term)


output.register_callback('notebook.lucky', lucky)





#--------------------------------History Screen----------------------------------


import requests

DB_URL = "https://razicloudlearn-default-rtdb.europe-west1.firebasedatabase.app/"

def get_user_history_by_username(username):
    # Construct the URL to fetch the users' data from Firebase
    users_url = f"{DB_URL}/users.json"

    # Fetch the users' data from Firebase
    response = requests.get(users_url)
    if response.ok:
        users_data = response.json()

        # Find the user by username
        for user_id, user_data in users_data.items():
            if user_data["username"] == username:
                # Return the user's history
                return user_data.get("history", [])

        else:
            print("Username not found in database.")
            return []

    else:
        print("Failed to fetch users' data from the database.")
        return []


def history_page():
    # Fetch user history using the existing function or logic
    user_history = get_user_history_by_username(valid_username)[::-1]

    # Process the history, skipping the first entry if it's a string
    processed_user_history = [entry for entry in user_history if isinstance(entry, dict)]

    if processed_user_history:
        # Generate HTML for the history page
        html_code  = f"""
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>History Page</title>
            <style>
                body {{
                    font-family: Arial, sans-serif;
                    margin: 20px;
                }}

                header {{
                    text-align: center;
                    padding: 10px;
                    background-color: #f2f2f2;
                }}

                section {{
                    margin-top: 20px;
                }}

                .event {{
                    border: 1px solid #ccc;
                    padding: 10px;
                    margin-bottom: 10px;
                }}

                footer {{
                    text-align: center;
                    padding: 10px;
                    background-color: #f2f2f2;
                }}
            </style>
        </head>
        <body>
            <header>
            <a href="#" onclick=" goBack(event)" class="back-icon">
            <img src="https://img.icons8.com/ios/452/circled-left-2.png" alt="Back" width="30" height="30"> </a>
            <h1>History Page</h1>
            </header>

            <section>
                {"".join(f'<div class="event"><p><strong>Search Word:</strong> {entry.get("query", "")}</p><p><strong>Date and Time:</strong> {entry.get("datetime", "")}</p></div>' for entry in processed_user_history)}
            </section>

            <footer>
                <p>&copy; 2024 History Page. All rights reserved.</p>
            </footer>
             <script>
        function goBack(event) {{
            event.preventDefault();
            google.colab.kernel.invokeFunction('notebook.go_to_search', [], {{}});
        }}
    </script>
        </body>

        </html>

        """
        return html_code
    else:
        return "No valid history found."

###########################################################################################################################




#--------------------------------ChatBot Interact----------------------------------
import json

patterns = create_patterns_from_data(DB_URL) # get all patterns

chatbot = Chat(patterns, reflections)

def pipeMessageFromUserIntoChatBot(userMessage):
  userMessage = userMessage.lower()
  response = chatbot.respond(userMessage)
  message_format = 'bot'
  if not response:
    return

  if len(response) <= 40:
    js_code = f'''
    send_message(`{response}`, `{message_format}`)
    '''
  else:
    response = json.loads(response)
    js_code = f'''
    robotResponse(`{response['url']}`, `{response['message_for_url']}`, `{response['header_message']}`, `{response['paragraph_message']}`)
    '''


  display(Javascript(js_code))

# response = chatbot.respond(user_input)
output.register_callback('notebook.pipeMessageFromUserIntoChatBot', pipeMessageFromUserIntoChatBot)



###########################################################################################################################

def go_to_search():
    # Clear the current output and display the login screen
    clear_output(wait=True)
    display(IPython.display.HTML(search_bar_html))
    display(IPython.display.HTML(search_results_html))
    display(IPython.display.HTML(htmlContentForChatBot))

output.register_callback('notebook.go_to_search', go_to_search)






#--------------------------------Admin Screen----------------------------------



URLs for cloud,Frequency,Action
https://www.oracle.com/il-en/corporate/cloud-compliance/,159,Remove
https://www.oracle.com/il-en/cloud/migrate-custom-applications-to-cloud/,137,Remove
https://www.oracle.com/il-en/cloud/migrate-isv-applications-to-oracle-cloud/,114,Remove
https://www.oracle.com/il-en/cloud/public-cloud-regions/,112,Remove
https://www.oracle.com/il-en/cloud/compute/vmware/,110,Remove
https://www.oracle.com/il-en/cloud/hybrid-cloud/,104,Remove
https://www.oracle.com/il-en/cloud/,93,Remove
https://www.oracle.com/il-en/cloud/workflow/,93,Remove
https://www.oracle.com/il-en/security/cloud-security/casb-cloud/,92,Remove
https://www.oracle.com/il-en/security/cloud-security/,91,Remove


# **Admin Screen**

# **Statistics Screen**